In [1]:
using ControlSystems
using DistributedControlSystems
using RobustAndOptimalControl
using LinearAlgebra
using BlockArrays

[ Info: Precompiling DistributedControlSystems [d14b68a6-6005-4866-9a10-8ceb977a395a]


In [2]:
T = [
    1 2 3
    -1 2 -1
    3 2 1
]

3×3 Matrix{Int64}:
  1  2   3
 -1  2  -1
  3  2   1

In [3]:
eigvals(T)

3-element Vector{ComplexF64}:
               -2.0 + 0.0im
 3.0000000000000013 - 1.7320508075688776im
 3.0000000000000013 + 1.7320508075688776im

In [4]:
A = T * Float64[
    0 1 0
    1 -1 1
    0 0 0
] * inv(T)

eigvals(A)

3-element Vector{Float64}:
 -1.618033988749895
 -1.3010426069826053e-16
  0.6180339887498946

In [5]:
B = T * Float64[
    2 0
    0 1
    0 0
]
B2 = T * Float64[0, 0, 1]
Bs = [B, B2]

2-element Vector{Array{Float64}}:
 [2.0 2.0; -2.0 2.0; 6.0 2.0]
 [3.0, -1.0, 1.0]

In [6]:
C = [1. 0 0] * inv(T)
C2 = [0. 0 1] * inv(T)
Cs = [C, C2]

2-element Vector{Matrix{Float64}}:
 [-0.16666666666666666 -0.16666666666666669 0.3333333333333333]
 [0.3333333333333333 -0.16666666666666666 -0.16666666666666666]

In [7]:
sys = ss(A, B, C, 0)
poles(sys)

3-element Vector{Float64}:
  0.6180339887498946
 -1.3010426069826053e-16
 -1.618033988749895

In [8]:
ssd = StateSpaceDistributed(A, Bs, Cs)

StateSpaceDistributed{Matrix{Float64}, BlockMatrix{Float64, Matrix{Matrix{Float64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}, BlockMatrix{Float64, Matrix{Matrix{Float64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}, BlockMatrix{Float64, Matrix{Matrix{Float64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}}([0.25 -1.0 0.24999999999999994; 0.08333333333333337 -1.6666666666666667 0.08333333333333331; 0.41666666666666663 -0.33333333333333337 0.4166666666666667], [2.0 2.0 3.0; -2.0 2.0 -1.0; 6.0 2.0 1.0], [-0.16666666666666666 -0.16666666666666669 0.3333333333333333; 0.3333333333333333 -0.16666666666666666 -0.16666666666666666], [0.0 0.0 0.0; 0.0 0.0 0.0])

In [9]:
modes(ssd)

2-element Vector{Vector{ComplexF64}}:
 [0.0 + 0.0im]
 [-1.6180339887 + 0.0im, 0.6180339887 + 0.0im]

In [10]:
fixed_modes(ssd)

ComplexF64[]

In [11]:
ssd2 = StateSpaceDistributed(
    ssd.A,
    reverse(ssd.B, dims=1),
    reverse(ssd.C, dims=2),
    reverse(ssd.D, dims=(1, 2)),
)

StateSpaceDistributed{Matrix{Float64}, BlockMatrix{Float64, Matrix{Matrix{Float64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}, BlockMatrix{Float64, Matrix{Matrix{Float64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}, BlockMatrix{Float64, Matrix{Matrix{Float64}}, Tuple{BlockedUnitRange{Vector{Int64}}, BlockedUnitRange{Vector{Int64}}}}}([0.25 -1.0 0.24999999999999994; 0.08333333333333337 -1.6666666666666667 0.08333333333333331; 0.41666666666666663 -0.33333333333333337 0.4166666666666667], [6.0 2.0 1.0; -2.0 2.0 -1.0; 2.0 2.0 3.0], [0.3333333333333333 -0.16666666666666669 -0.16666666666666666; -0.16666666666666666 -0.16666666666666666 0.3333333333333333], [0.0 0.0 0.0; 0.0 0.0 0.0])

In [12]:
modes(ssd2)

2-element Vector{Vector{ComplexF64}}:
 [-1.6180339887 + 0.0im, 0.6180339887 + 0.0im]
 [-0.0 + 0.0im]

In [13]:
fixed_modes(ssd2)

ComplexF64[]

In [14]:
dc, Acl = distributed_controllers(ssd)

(StateSpace{Continuous, Float64}[StateSpace{Continuous, Float64}
A = 
 -8.624594279853605   -0.042210950932454194
 -0.6665250199379624  -2.925482550283489
B = 
 -5.6612084196982275
 -2.2670598784252296
C = 
 -1.044450047287927   -0.024475130695713405
 -0.2957503004589669  -0.6088329303475555
D = 
 0.0
 0.0

Continuous-time state-space model, StateSpace{Continuous, Float64}
A = 
 -2.8577380332470406
B = 
 -1.0
C = 
 -0.9999999999999999
D = 
 0.0

Continuous-time state-space model], [0.25 -1.0 … 1.2666161220865377 2.9999999999999996; 0.08333333333333337 -1.6666666666666667 … 1.1687155993036842 -0.9999999999999999; … ; 0.3778433130708716 0.37784331307087166 … -2.925482550283489 -0.0; -0.3333333333333333 0.16666666666666666 … 0.0 -2.8577380332470406])

In [15]:
dc2, Acl2 = distributed_controllers(ssd2)

(StateSpace{Continuous, Float64}[StateSpace{Continuous, Float64}
A = 
 -5.30722777603022
B = 
 1.0
C = 
  1.0
 -9.064933036736788e-17
D = 
 0.0
 0.0

Continuous-time state-space model, StateSpace{Continuous, Float64}
A = 
 -5.260483530696403     0.47480458403687653
  0.20717927064795694  -1.126870874592368
B = 
 -5.661208419698209
 -2.2670598784252203
C = 
 -1.2359008849938513  -0.22515490155571588
D = 
 0.0

Continuous-time state-space model], [0.25 -1.0 … 1.2359008849938513 0.22515490155571588; 0.08333333333333337 -1.6666666666666667 … -1.2359008849938513 -0.22515490155571588; … ; 0.9435347366163681 0.9435347366163681 … -5.260483530696403 0.47480458403687653; 0.37784331307087005 0.37784331307087005 … 0.20717927064795694 -1.126870874592368])

In [16]:
Acl |> eigvals

6-element Vector{Float64}:
 -6.912489081277625
 -3.349252857163486
 -2.449489742783178
 -1.6475369481638107
 -0.6407979435321783
 -0.4082482904638645

In [17]:
Acl2 |> eigvals

6-element Vector{Float64}:
 -4.898979485566361
 -3.256616537982939
 -1.8424029756098497
 -1.6475369481638085
 -0.6407979435321737
 -0.40824829046386557